In [1]:
# Getting Premier league data from 2020-2023 games

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [7]:
# Scraping through multiple seasons and teams
years = list(range(2023, 2022, -1))

all_matches = []
# URL where we are scraping data
standings_url = "https://fbref.com/en/comps/9/Premier-Leagues-Stats"

for year in years:
    # Download html document of the site 
    data = requests.get(standings_url)
    # Parse our html
    soup = BeautifulSoup(data.text)
    # Create a seletor for the beautiful soup to select
# Select the tabele from the page
    standings_table = soup.select('table.stats_table')[0]
    # Find all a tags for all teams in the table
    links = standings_table.find_all('a')
    # Find the href of all a-tags 
    links = [l.get('href') for l in links]
    #filter our links so that we have only squad links
    links = [l for l in links if '/squads/' in l]
    #filter our links so that we have only squad links
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")
        
        soup = BeautifulSoup(data.text) 
        links = soup.find_all('a')
        links = [l.get('href') for l in links]
        links = [l for l in links if  l and 'all_comps/shooting/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]
        data = requests.get(team_urls[0])
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = pd.merge(matches[0], shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], how='outer', on='Date')
            #matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on='Date')
        except ValueError:
            continue
        
        team_data["Season"] = year
        team_data["Team"] = team_name
            
        all_matches.append(team_data)
        time.sleep(1)
        
print("Done!!")   

Done!!


In [8]:
match_df = pd.concat(all_matches)
pd.options.display.max_rows = 9999
df = match_df[match_df['Comp'] == "Premier League"]
df.sort_values(by='Date')

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,...,Match Report,Arsenal won on penalty kicks following normal ...,7.0,3.0,NaN,NaN,0.0,0.0,2023,Arsenal
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0.0,0.0,2023,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1.0,1.0,2023,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1.0,1.0,2023,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0.0,0.0,2023,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1,0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0.0,0.0,2023,Arsenal
6,2023-09-20,20:00,Champions Lg,Group stage,Wed,Home,W,4,0,nl PSV Eindhoven,...,Match Report,NaN,18.0,8.0,14.4,0.0,0.0,0.0,2023,Arsenal
7,2023-09-24,14:00,Premier League,Matchweek 6,Sun,Home,D,2,2,Tottenham,...,Match Report,NaN,12.0,4.0,16.9,0.0,1.0,1.0,2023,Arsenal
8,2023-09-27,19:45,EFL Cup,Third round,Wed,Away,W,1,0,Brentford,...,Match Report,NaN,10.0,3.0,NaN,NaN,0.0,0.0,2023,Arsenal
9,2023-09-30,15:00,Premier League,Matchweek 7,Sat,Away,W,4,0,Bournemouth,...,Match Report,NaN,13.0,6.0,15.5,0.0,2.0,2.0,2023,Arsenal


In [ ]:
match_df.to_csv("Premier_League_Data.csv")